# TimesFM 投资组合优化 - 简化版

本notebook使用重构后的工具类，代码更加简洁易懂。

## 主要改进

- ✅ 数据处理工具模块化
- ✅ 模型定义清晰分离
- ✅ 训练过程自动化
- ✅ 可视化工具统一管理
- ✅ 配置参数集中管理

## 1. 环境设置和导入

## 🔧 快速测试模式说明

本notebook支持两种运行模式：

### ⚡ 快速测试模式 (`QUICK_TEST_MODE = True`)
- **数据量**: 500个时间点（约2年数据）
- **时间窗口**: 100个时间步
- **批量大小**: 32
- **训练轮次**: 3轮
- **预计时间**: 5-10分钟
- **用途**: 快速验证代码逻辑，调试流程

### 🚀 完整训练模式 (`QUICK_TEST_MODE = False`)
- **数据量**: 完整数据集
- **时间窗口**: 200个时间步
- **批量大小**: 128
- **训练轮次**: 20轮
- **预计时间**: 1-3小时
- **用途**: 完整模型训练，获得最佳性能

💡 **建议**: 首次运行时使用快速测试模式验证流程，确认无误后再切换到完整模式进行正式训练。

## ⚙️ 统一配置参数

所有可配置的参数都集中在这里，方便统一设置和管理。

In [ ]:
# 🎛️ 全局配置参数中心
# ============================================================================
# 所有可配置的参数都在这里，修改后会影响整个notebook的运行行为
# ============================================================================

# 📊 运行模式配置
# ============================================================================
QUICK_TEST_MODE = True  # 设置为False使用完整数据集和完整训练
QUICK_TEST_DATA_SIZE = 150  # 快速测试模式下使用的数据量

# 🔧 数据处理配置  
# ============================================================================
# 特征选择 - 可选项: 'price_only', 'price_and_volume', 'close_only', 'ohlc', 'close_and_volume'
SELECTED_FEATURES = 'close_only'  

# 数据分割比例
TRAIN_RATIO = 0.8  # 训练集占比
VAL_RATIO = 0.8    # 验证集占训练集的比例

# 🏗️ 模型架构配置
# ============================================================================
# 基础模型配置
FINETUNE_TIMESFM_BASIC = False  # 基础模型是否微调TimesFM

# 高级注意力模型配置  
ATTENTION_HEADS = 8        # 多头注意力的头数
ATTENTION_HIDDEN_DIM = 256 # 注意力网络隐藏层维度
MAX_SINGLE_WEIGHT = 0.25   # 单个资产的最大权重限制

# TimesFM配置
TIMESFM_CONTEXT_LEN = 200  # TimesFM的上下文长度（完整模式）
TIMESFM_CONTEXT_LEN_QUICK = 50  # 快速测试模式的上下文长度

# 📚 数据加载配置
# ============================================================================
# 批量大小配置
BATCH_SIZE_QUICK = 32   # 快速测试模式批量大小
BATCH_SIZE_FULL = 256   # 完整模式批量大小

# 数据加载器配置
NUM_WORKERS = 4         # 数据加载进程数
PIN_MEMORY = True       # 是否使用锁页内存
PERSISTENT_WORKERS = True # 是否使用持久化工作进程

# 🏋️ 训练配置
# ============================================================================
# 基础模型训练配置
BASIC_MODEL_EPOCHS_QUICK = 3   # 快速测试模式训练轮次
BASIC_MODEL_EPOCHS_FULL = 20   # 完整模式训练轮次
BASIC_MODEL_PATIENCE = 15      # 早停耐心值

# 高级模型训练配置  
ADVANCED_MODEL_EPOCHS_QUICK = 5   # 快速测试模式训练轮次
ADVANCED_MODEL_EPOCHS_FULL = 50   # 完整模式训练轮次  
ADVANCED_MODEL_PATIENCE = 15      # 早停耐心值

# 学习率配置
PORTFOLIO_LEARNING_RATE = 1e-4    # 投资组合层学习率
TIMESFM_LEARNING_RATE = 1e-5      # TimesFM微调学习率
TIMESFM_FINETUNE_LR = 1e-6        # 第二阶段微调学习率

# 📉 损失函数配置
# ============================================================================
RISK_AVERSION = 1.0        # 风险厌恶系数
SHARPE_WEIGHT = 0.5        # 夏普比率权重
RETURN_WEIGHT = 0.5        # 收益权重

# 💾 模型保存配置
# ============================================================================
MODEL_SAVE_DIR = 'models'                    # 模型保存目录
CHECKPOINT_INTERVAL = 5                      # 检查点保存间隔
MAX_CHECKPOINTS = 5                          # 最大检查点数量
MAX_BEST_MODELS = 3                          # 最大最佳模型数量

# 🎨 可视化配置
# ============================================================================
FIGURE_DPI = 100           # 图像DPI
FIGURE_SIZE = (15, 10)     # 默认图像大小
SAVE_PLOTS = False         # 是否保存图像到文件
# 字体配置
CHINESE_FONTS = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS', 'Noto Sans CJK SC', 'WenQuanYi Micro Hei']
SUPPRESS_FONT_WARNINGS = True  # 是否抑制字体警告

# 🔍 调试配置
# ============================================================================
VERBOSE_MODE = True        # 是否显示详细日志
GRAD_CLIP_VALUE = 1.0      # 梯度裁剪值
RANDOM_SEED = 42           # 随机种子

# ============================================================================
# 配置验证和预处理
# ============================================================================

# 验证配置合理性
def validate_config():
    """验证配置参数的合理性"""
    issues = []
    
    if not 0 < TRAIN_RATIO <= 1:
        issues.append(f"TRAIN_RATIO 应该在(0,1]范围内，当前值: {TRAIN_RATIO}")
    
    if not 0 < VAL_RATIO <= 1:
        issues.append(f"VAL_RATIO 应该在(0,1]范围内，当前值: {VAL_RATIO}")
        
    if not 0 < MAX_SINGLE_WEIGHT <= 1:
        issues.append(f"MAX_SINGLE_WEIGHT 应该在(0,1]范围内，当前值: {MAX_SINGLE_WEIGHT}")
        
    if ATTENTION_HEADS <= 0:
        issues.append(f"ATTENTION_HEADS 应该大于0，当前值: {ATTENTION_HEADS}")
        
    if issues:
        print("⚠️  配置验证发现问题:")
        for issue in issues:
            print(f"   - {issue}")
        return False
    
    print("✅ 配置参数验证通过")
    return True

# 根据运行模式设置动态参数
def setup_dynamic_config():
    """根据QUICK_TEST_MODE设置相应的参数"""
    global window_size, batch_size, basic_epochs, advanced_epochs
    
    if QUICK_TEST_MODE:
        window_size = TIMESFM_CONTEXT_LEN_QUICK
        batch_size = BATCH_SIZE_QUICK
        basic_epochs = BASIC_MODEL_EPOCHS_QUICK
        advanced_epochs = ADVANCED_MODEL_EPOCHS_QUICK
        print(f"⚡ 快速测试模式配置:")
        print(f"   数据量: {QUICK_TEST_DATA_SIZE}")
        print(f"   时间窗口: {window_size}")
        print(f"   批量大小: {batch_size}")
        print(f"   基础模型轮次: {basic_epochs}")
        print(f"   高级模型轮次: {advanced_epochs}")
    else:
        window_size = TIMESFM_CONTEXT_LEN
        batch_size = BATCH_SIZE_FULL
        basic_epochs = BASIC_MODEL_EPOCHS_FULL
        advanced_epochs = ADVANCED_MODEL_EPOCHS_FULL
        print(f"🚀 完整训练模式配置:")
        print(f"   时间窗口: {window_size}")
        print(f"   批量大小: {batch_size}")
        print(f"   基础模型轮次: {basic_epochs}")
        print(f"   高级模型轮次: {advanced_epochs}")

# 配置matplotlib中文字体 - 增强版
def setup_matplotlib_fonts():
    """配置matplotlib中文字体支持 - 增强版配置"""
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import warnings
    
    try:
        # 清除matplotlib字体缓存
        mpl.font_manager._rebuild()
        
        # 强制设置字体
        plt.rcParams['font.family'] = ['sans-serif']
        plt.rcParams['font.sans-serif'] = CHINESE_FONTS
        plt.rcParams['axes.unicode_minus'] = False
        
        # 设置图像参数
        plt.rcParams['figure.dpi'] = FIGURE_DPI
        plt.rcParams['figure.figsize'] = FIGURE_SIZE
        
        # 抑制字体相关警告（如果配置了的话）
        if SUPPRESS_FONT_WARNINGS:
            warnings.filterwarnings('ignore', message='.*Glyph.*missing from font.*')
            warnings.filterwarnings('ignore', category=mpl.font_manager.FontProperties)
            print("🔇 已抑制matplotlib字体警告信息")
        
        print("✅ matplotlib增强字体配置完成")
        print(f"   主字体: {CHINESE_FONTS[0]}")
        print(f"   备选字体: {', '.join(CHINESE_FONTS[1:3])}")
        return True
    except Exception as e:
        print(f"⚠️  matplotlib配置警告: {e}")
        print("   将使用默认设置")
        return False

# 执行配置设置
validate_config()
setup_dynamic_config()
setup_matplotlib_fonts()

print("\n" + "="*60)
print("🎛️ 配置参数中心设置完成")
print("💡 如需修改参数，请在此单元格中调整相应变量")
print("🎨 matplotlib中文字体支持已配置（警告已抑制）")
print("="*60)

### 📝 使用说明

**配置完成后，您可以：**

1. **快速开始**: 保持默认配置直接运行所有单元格
2. **调整模式**: 修改 `QUICK_TEST_MODE` 切换快速测试/完整训练
3. **自定义参数**: 根据需求调整任何配置参数
4. **批量修改**: 所有参数在一处管理，修改方便

**重要提醒**: 修改配置后，请重新运行本单元格以应用新配置。

## ✅ 配置整合完成总结

### 🎯 已完成的配置整合工作：

1. **创建统一配置中心** - 所有可配置参数现在集中在一个地方
2. **参数分类组织** - 按功能分组（运行模式、数据处理、模型架构、训练等）
3. **配置验证** - 自动检查参数合理性，防止配置错误
4. **动态配置** - 根据运行模式自动设置相关参数
5. **更新引用** - notebook中的相关代码已更新为使用统一配置

### 📊 主要配置类别：

- **🔧 运行模式**: 快速测试/完整训练模式切换
- **📈 数据处理**: 特征选择、数据分割、批量大小等
- **🏗️ 模型架构**: TimesFM配置、注意力机制参数
- **🏋️ 训练设置**: 学习率、训练轮次、早停参数  
- **📉 损失函数**: 风险厌恶、权重平衡
- **💾 模型保存**: 检查点管理、目录配置
- **🎨 可视化**: 图像设置、保存选项

### 💡 使用建议：

- **首次运行**: 使用默认配置即可快速开始
- **性能调优**: 根据硬件调整批量大小和工作进程数
- **实验对比**: 修改风险厌恶系数观察策略变化
- **生产部署**: 切换到完整模式获得最佳性能

现在整个notebook的配置管理更加清晰和便捷了！

In [ ]:
# 📦 导入必要的库
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import warnings
import gc
from typing import Dict, List, Tuple, Optional, Any
import os
import sys

# 🔧 配置matplotlib中文字体支持
try:
    # 尝试设置中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS', 'Noto Sans CJK SC']
    plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
    print("✅ matplotlib中文字体配置成功")
except Exception as e:
    print(f"⚠️  中文字体配置警告: {e}")
    print("   将使用默认英文字体，可能出现中文显示问题")

# 🔧 其他配置
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# 设置随机种子
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 🖥️ 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 使用设备: {device}")

if device.type == 'cuda':
    print(f"   GPU名称: {torch.cuda.get_device_name(0)}")
    print(f"   GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # 优化GPU内存使用
    torch.backends.cudnn.benchmark = True
else:
    print("   使用CPU运算模式")

print(f"📊 PyTorch版本: {torch.__version__}")
print(f"📊 NumPy版本: {np.__version__}")
print(f"📊 Pandas版本: {pd.__version__}")

# 确保xiaojiucai包路径
project_root = Path('/home/userroot/dev/xiaojiucai')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ 已添加项目路径: {project_root}")

print("🎉 环境配置完成！")

# 导入可视化相关函数
from xiaojiucai.visualize import plot_training_curves, setup_matplotlib_chinese, create_comprehensive_report

# 设置中文显示
setup_matplotlib_chinese()

## 2. 数据处理

In [ ]:
# 📊 导入数据处理相关模块
from xiaojiucai.data import DataProcessor, print_feature_configs, FEATURE_CONFIGS

# 查看可用的特征配置
print_feature_configs()

# 🔧 快速测试模式配置 (现在使用统一配置中心的参数)
print(f"\n🚀 运行模式: {'快速测试模式' if QUICK_TEST_MODE else '完整数据模式'}")
if QUICK_TEST_MODE:
    print(f"📊 测试数据量: {QUICK_TEST_DATA_SIZE} 个时间点")
    print(f"📊 选择特征: {SELECTED_FEATURES}")

# 创建数据处理器
selected_features = FEATURE_CONFIGS[SELECTED_FEATURES]  # 使用统一配置的特征选择
processor = DataProcessor(
    selected_features=selected_features,
    test_mode=QUICK_TEST_MODE,  # 使用快速测试模式
    test_data_size=QUICK_TEST_DATA_SIZE
)

# 加载和预处理数据
features_data, returns, tickers = processor.load_and_preprocess('../data/mag7_data_raw.parquet')

print(f"\n✅ 数据处理完成:")
print(f"  特征数据形状: {features_data.shape}")
print(f"  收益数据形状: {returns.shape}")
print(f"  股票列表: {tickers}")
print(f"  数据时间跨度: {features_data.shape[0]} 个交易日")

In [ ]:
# 📊 快速测试模式：可视化测试集数据
if QUICK_TEST_MODE:
    import matplotlib.pyplot as plt
    import pandas as pd
    
    print(f"\n📈 绘制测试集收盘价数据 (最近{QUICK_TEST_DATA_SIZE}个交易日)")
    
    # 创建时间索引（假设是最近的交易日）
    dates = pd.date_range(end='2025-09-25', periods=features_data.shape[0], freq='B')
    
    # 创建图形
    plt.figure(figsize=(15, 10))
    
    # 绘制每只股票的收盘价
    for i, ticker in enumerate(tickers):
        # features_data的第三维是特征维度，索引0是Close价格
        close_prices = features_data[:, i, 0]  # (时间步, 股票i, Close特征)
        plt.subplot(2, 4, i+1)
        plt.plot(dates, close_prices, linewidth=2, color=f'C{i}')
        plt.title(f'{ticker} 收盘价走势', fontsize=12, fontweight='bold')
        plt.xlabel('日期', fontsize=10)
        plt.ylabel('价格', fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        
        # 显示统计信息
        avg_price = np.mean(close_prices)
        min_price = np.min(close_prices)
        max_price = np.max(close_prices)
        plt.text(0.02, 0.98, f'均价: ${avg_price:.1f}\n最低: ${min_price:.1f}\n最高: ${max_price:.1f}', 
                transform=plt.gca().transAxes, verticalalignment='top', fontsize=9,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.suptitle(f'MAG7 股票收盘价走势 - 快速测试数据集\n({dates[0].strftime("%Y-%m-%d")} 至 {dates[-1].strftime("%Y-%m-%d")})', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    # 显示数据范围统计
    print(f"\n📊 测试集数据统计:")
    print(f"  数据时间范围: {dates[0].strftime('%Y-%m-%d')} 至 {dates[-1].strftime('%Y-%m-%d')}")
    print(f"  总交易日数: {len(dates)}")
    
    # 价格水平统计
    all_prices = features_data[:, :, 0].flatten()
    print(f"  整体价格统计:")
    print(f"    平均价格: ${np.mean(all_prices):.2f}")
    print(f"    价格范围: ${np.min(all_prices):.2f} - ${np.max(all_prices):.2f}")
    print(f"    标准差: ${np.std(all_prices):.2f}")
    
    # 各股票价格对比
    print(f"\n💰 各股票平均价格对比:")
    for i, ticker in enumerate(tickers):
        avg_price = np.mean(features_data[:, i, 0])
        print(f"    {ticker}: ${avg_price:.2f}")
        
    print(f"\n" + "="*50)

## 3. 创建训练数据

In [ ]:
# 📊 导入数据处理函数
from xiaojiucai.data import prepare_data_tensors, create_data_loaders

# 🔧 使用统一配置中心的参数
print(f"⚡ 当前数据处理参数:")
print(f"  时间窗口: {window_size}")
print(f"  批量大小: {batch_size}")
print(f"  训练比例: {TRAIN_RATIO}")
print(f"  验证比例: {VAL_RATIO}")
if QUICK_TEST_MODE:
    print(f"  数据量限制: {QUICK_TEST_DATA_SIZE}")

# 准备训练数据张量
print("🔄 准备训练数据张量...")
data_dict = prepare_data_tensors(
    features_data, returns, window_size,
    train_ratio=TRAIN_RATIO, val_ratio=VAL_RATIO, device=device
)
print("✅ 数据张量准备完成")

# 创建数据加载器 - 使用兼容的参数
print("🔄 创建数据加载器...")
train_loader, val_loader, test_loader = create_data_loaders(
    data_dict, 
    batch_size=batch_size, 
    num_workers=min(NUM_WORKERS, 2),  # 限制worker数量避免问题
    pin_memory=PIN_MEMORY
)
print("✅ 数据加载器创建完成")

print(f"\n📊 数据集划分:")
print(f"  训练集: {len(train_loader.dataset)} 样本 ({len(train_loader)} 批次)")
print(f"  验证集: {len(val_loader.dataset)} 样本 ({len(val_loader)} 批次)")
print(f"  测试集: {len(test_loader.dataset)} 样本 ({len(test_loader)} 批次)")
print(f"  数据加载器配置: batch_size={batch_size}, num_workers={min(NUM_WORKERS, 2)}, pin_memory={PIN_MEMORY}")

In [ ]:
# 📊 快速测试模式：可视化收益率分布
if QUICK_TEST_MODE:
    print(f"\n📈 可视化收益率分布和相关性")
    
    # 创建收益率可视化
    plt.figure(figsize=(16, 12))
    
    # 1. 收益率时间序列
    plt.subplot(2, 3, 1)
    for i, ticker in enumerate(tickers):
        returns_series = data_dict['y_test'][:, i].cpu().numpy()
        plt.plot(returns_series[:50], label=ticker, alpha=0.7)  # 只显示前50个点避免过于拥挤
    plt.title('测试集收益率时间序列 (前50个样本)', fontweight='bold')
    plt.xlabel('样本序号')
    plt.ylabel('收益率')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. 收益率分布直方图
    plt.subplot(2, 3, 2)
    all_returns = data_dict['y_test'].cpu().numpy().flatten()
    plt.hist(all_returns, bins=50, alpha=0.7, edgecolor='black')
    plt.title('收益率分布直方图', fontweight='bold')
    plt.xlabel('收益率')
    plt.ylabel('频次')
    plt.grid(True, alpha=0.3)
    
    # 显示统计信息
    plt.text(0.02, 0.98, f'均值: {np.mean(all_returns):.4f}\n标准差: {np.std(all_returns):.4f}\n偏度: {np.mean((all_returns-np.mean(all_returns))**3)/np.std(all_returns)**3:.4f}', 
             transform=plt.gca().transAxes, verticalalignment='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 3. 收益率箱线图
    plt.subplot(2, 3, 3)
    returns_by_stock = [data_dict['y_test'][:, i].cpu().numpy() for i in range(len(tickers))]
    box_plot = plt.boxplot(returns_by_stock, labels=tickers, patch_artist=True)
    plt.title('各股票收益率分布', fontweight='bold')
    plt.xlabel('股票')
    plt.ylabel('收益率')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # 为箱线图添加颜色
    colors = plt.cm.Set3(np.linspace(0, 1, len(tickers)))
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
    
    # 4. 相关性热力图
    plt.subplot(2, 3, 4)
    returns_corr = np.corrcoef(data_dict['y_test'].cpu().numpy().T)
    im = plt.imshow(returns_corr, cmap='RdYlBu_r', vmin=-1, vmax=1)
    plt.colorbar(im, shrink=0.8)
    plt.title('收益率相关性矩阵', fontweight='bold')
    plt.xticks(range(len(tickers)), tickers, rotation=45)
    plt.yticks(range(len(tickers)), tickers)
    
    # 在热力图上显示数值
    for i in range(len(tickers)):
        for j in range(len(tickers)):
            plt.text(j, i, f'{returns_corr[i,j]:.2f}', 
                    ha='center', va='center', fontsize=8)
    
    # 5. 波动率分析
    plt.subplot(2, 3, 5)
    volatilities = [np.std(data_dict['y_test'][:, i].cpu().numpy()) for i in range(len(tickers))]
    bars = plt.bar(tickers, volatilities, color=colors)
    plt.title('各股票收益率波动率', fontweight='bold')
    plt.xlabel('股票')
    plt.ylabel('波动率 (标准差)')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # 在柱状图上显示数值
    for bar, vol in zip(bars, volatilities):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001, 
                f'{vol:.4f}', ha='center', va='bottom', fontsize=9)
    
    # 6. 累积收益曲线
    plt.subplot(2, 3, 6)
    for i, ticker in enumerate(tickers):
        returns_series = data_dict['y_test'][:, i].cpu().numpy()
        cumulative_returns = np.cumsum(returns_series)
        plt.plot(cumulative_returns, label=ticker, alpha=0.8)
    plt.title('累积收益曲线', fontweight='bold')
    plt.xlabel('样本序号')
    plt.ylabel('累积收益')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle(f'快速测试模式 - 收益率数据分析\n测试集样本数: {len(data_dict["y_test"])}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    # 打印详细统计信息
    print(f"\n📊 收益率统计摘要:")
    print(f"  测试集样本数: {len(data_dict['y_test'])}")
    print(f"  股票数量: {len(tickers)}")
    print(f"  整体收益率统计:")
    print(f"    均值: {np.mean(all_returns):.6f}")
    print(f"    标准差: {np.std(all_returns):.6f}")
    print(f"    最小值: {np.min(all_returns):.6f}")
    print(f"    最大值: {np.max(all_returns):.6f}")
    
    print(f"\n💹 各股票收益率特征:")
    for i, ticker in enumerate(tickers):
        stock_returns = data_dict['y_test'][:, i].cpu().numpy()
        print(f"  {ticker}:")
        print(f"    均值: {np.mean(stock_returns):.6f}")
        print(f"    波动率: {np.std(stock_returns):.6f}")
        print(f"    夏普比率: {np.mean(stock_returns)/np.std(stock_returns):.6f}")
    
    print(f"\n" + "="*50)

## 4. 模型创建

## 4.5 多层注意力Head实验

基于之前的讨论，我们将实验一个更复杂的投资组合头网络，使用多层注意力机制来捕捉：
- 资产间的复杂关系
- 时间序列的动态依赖
- 风险和收益的联合建模

这个方法避免直接预测股价，而是专注于学习资产配置的复杂模式。

In [ ]:
# 🧠 高级多层注意力投资组合头网络
import torch
import numpy as np

class AdvancedAttentionPortfolioHead(torch.nn.Module):
    """
    高级多层注意力投资组合头网络
    
    特点:
    1. 多头注意力捕捉资产间关系
    2. 时间注意力建模时间依赖
    3. 分别建模期望收益和风险协方差
    4. 集成权重生成和多样化约束
    """
    
    def __init__(self, timesfm_dim, num_assets, hidden_dim=256, num_heads=8, max_weight=0.3):
        super().__init__()
        self.num_assets = num_assets
        self.timesfm_dim = timesfm_dim
        self.hidden_dim = hidden_dim
        self.max_weight = max_weight
        
        # 多头注意力 - 捕捉资产间关系
        self.cross_asset_attention = torch.nn.MultiheadAttention(
            embed_dim=timesfm_dim, 
            num_heads=num_heads, 
            batch_first=True,
            dropout=0.1
        )
        
        # 时间注意力 - 捕捉时间依赖
        self.temporal_attention = torch.nn.MultiheadAttention(
            embed_dim=timesfm_dim,
            num_heads=num_heads,
            batch_first=True,
            dropout=0.1
        )
        
        # 特征提取网络
        self.feature_extractor = torch.nn.Sequential(
            torch.nn.Linear(timesfm_dim, hidden_dim),
            torch.nn.LayerNorm(hidden_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            
            torch.nn.Linear(hidden_dim, hidden_dim // 2),
            torch.nn.LayerNorm(hidden_dim // 2),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
        )
        
        # 期望收益预测器
        self.return_predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets),
            torch.nn.Tanh()  # 限制预测范围
        )
        
        # 风险估计器 - 预测协方差矩阵的因子
        self.risk_factor_predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets * 3),  # 3个风险因子
        )
        
        # 特质风险预测器
        self.idiosyncratic_risk = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, num_assets),
            torch.nn.Softplus()  # 确保正值
        )
        
        # 市场状态分类器
        self.market_state_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, 32),
            torch.nn.GELU(),
            torch.nn.Linear(32, 3),  # 3种市场状态：牛市、熊市、震荡
            torch.nn.Softmax(dim=-1)
        )
        
        # 权重生成网络 - 集成所有信息
        self.weight_generator = torch.nn.Sequential(
            torch.nn.Linear(num_assets + 3, hidden_dim // 2),  # 期望收益 + 市场状态
            torch.nn.LayerNorm(hidden_dim // 2),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets)
        )
        
        # 权重约束层
        self.softmax_temp = torch.nn.Parameter(torch.tensor(2.0))  # 可学习的温度参数
        
    def forward(self, timesfm_predictions):
        """
        timesfm_predictions: (batch_size, num_assets, timesfm_dim) 
        - 每个资产的TimesFM预测特征
        """
        batch_size, num_assets, feature_dim = timesfm_predictions.shape
        
        # 1. 跨资产注意力 - 学习资产间关系
        asset_features, asset_attention = self.cross_asset_attention(
            timesfm_predictions, timesfm_predictions, timesfm_predictions
        )
        
        # 2. 时间注意力 - 如果有时间维度的话（这里简化为资产维度）
        temporal_features, temporal_attention = self.temporal_attention(
            asset_features, asset_features, asset_features
        )
        
        # 3. 特征聚合 - 每个资产的综合特征
        aggregated_features = torch.mean(temporal_features, dim=1)  # (batch_size, feature_dim)
        
        # 4. 特征提取
        extracted_features = self.feature_extractor(aggregated_features)
        
        # 5. 期望收益预测
        expected_returns = self.return_predictor(extracted_features)
        
        # 6. 风险建模
        risk_factors = self.risk_factor_predictor(extracted_features)
        risk_factors = risk_factors.view(batch_size, num_assets, 3)  # (batch, assets, factors)
        
        # 协方差矩阵 = 因子模型: Σ = B @ B^T + D
        # 其中 B 是因子载荷矩阵，D 是特质风险
        factor_covariance = torch.bmm(risk_factors, risk_factors.transpose(-1, -2))
        idiosyncratic_risks = self.idiosyncratic_risk(extracted_features)
        diagonal_risk = torch.diag_embed(idiosyncratic_risks)
        estimated_covariance = factor_covariance + diagonal_risk
        
        # 7. 市场状态识别
        market_states = self.market_state_classifier(extracted_features)
        
        # 8. 综合权重生成
        # 结合期望收益和市场状态信息
        combined_features = torch.cat([expected_returns, market_states], dim=-1)
        raw_weights = self.weight_generator(combined_features)
        
        # 9. 权重约束和归一化
        # 使用可学习的温度参数进行Softmax
        constrained_weights = torch.softmax(raw_weights / self.softmax_temp, dim=-1)
        
        # 10. 最大权重约束
        constrained_weights = torch.clamp(constrained_weights, max=self.max_weight)
        final_weights = constrained_weights / torch.sum(constrained_weights, dim=1, keepdim=True)
        
        return final_weights, {
            'expected_returns': expected_returns,
            'covariance_matrix': estimated_covariance,
            'market_states': market_states,
            'asset_attention': asset_attention,
            'temporal_attention': temporal_attention,
            'risk_factors': risk_factors,
            'idiosyncratic_risks': idiosyncratic_risks
        }

# 创建使用高级注意力Head的投资组合模型
class TimesFMAdvancedPortfolioModel(torch.nn.Module):
    """集成TimesFM和高级注意力Head的投资组合模型"""
    
    def __init__(self, input_size, output_size, feature_size, 
                 timesfm_wrapper, context_len, finetune_timesfm=False):
        super().__init__()
        self.timesfm_wrapper = timesfm_wrapper
        self.timesfm_model = timesfm_wrapper.model
        self.context_len = context_len
        self.feature_size = feature_size
        self.finetune_timesfm = finetune_timesfm
        
        # 设置TimesFM参数的梯度
        for param in self.timesfm_model.parameters():
            param.requires_grad = finetune_timesfm
            
        if finetune_timesfm:
            print("🔓 TimesFM参数已解冻，将进行端到端微调")
        else:
            print("🔒 TimesFM参数已冻结，仅训练投资组合优化层")
            
        # 特征处理层
        self.feature_processor = torch.nn.Sequential(
            torch.nn.Linear(feature_size, 64),
            torch.nn.LayerNorm(64),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(64, 32),
            torch.nn.LayerNorm(32),
            torch.nn.GELU(),
        )
        
        # 高级注意力投资组合头
        self.portfolio_head = AdvancedAttentionPortfolioHead(
            timesfm_dim=32,  # 与feature_processor输出维度匹配
            num_assets=output_size,
            hidden_dim=256,
            num_heads=8,
            max_weight=0.25  # 单个资产最大25%权重
        )

    def forward(self, x):
        # x的形状: (batch_size, context_len, num_assets, num_features)
        batch_size, _, num_assets, num_features = x.shape
        forecast_horizon = 1
        
        # 处理每个资产的特征
        processed_features = []
        timesfm_predictions = []
        
        for i in range(num_assets):
            # 提取单个资产的特征
            asset_features = x[:, :, i, :]  # (batch_size, context_len, num_features)
            
            # 特征处理
            asset_features_flat = asset_features.view(-1, num_features)
            processed = self.feature_processor(asset_features_flat)
            processed = processed.view(batch_size, self.context_len, -1)
            processed_features.append(processed)
            
            # TimesFM预测（如果需要）
            # 这里我们将处理后的特征聚合为时间序列
            asset_series_1d = torch.mean(processed, dim=2)  # (batch_size, context_len)
            masks = torch.zeros_like(asset_series_1d, dtype=torch.bool)
            
            # 获取TimesFM特征表示（而不是预测结果）
            if self.training and self.finetune_timesfm:
                # 使用梯度模式获取特征
                with torch.set_grad_enabled(True):
                    forecasts, _, _ = self.timesfm_wrapper.decode_with_grad(
                        forecast_horizon, asset_series_1d, masks, enable_grad=True
                    )
                    # 使用最后一层的隐藏状态作为特征
                    timesfm_features = forecasts[:, -1, 0, :5]  # 取前5个分位数作为特征
            else:
                # 推理模式：使用处理后的特征作为近似
                timesfm_features = torch.mean(processed, dim=1)  # (batch_size, 32)
            
            timesfm_predictions.append(timesfm_features)
        
        # 堆叠所有资产的TimesFM特征
        stacked_predictions = torch.stack(timesfm_predictions, dim=1)  # (batch_size, num_assets, feature_dim)
        
        # 通过高级注意力头生成权重
        weights, aux_outputs = self.portfolio_head(stacked_predictions)
        
        return weights, aux_outputs

print("✅ 高级多层注意力投资组合模型定义完成！")

In [ ]:
# 🎯 高级模型训练配置
def create_advanced_attention_model():
    """创建使用高级注意力机制的投资组合模型"""
    
    print("🔧 创建高级多层注意力投资组合模型...")
    
    # 检查必要的变量是否存在，如果不存在则使用默认值
    if 'data_dict' in globals():
        context_len = data_dict['window_size']
        num_features = data_dict['X_train'].shape[-1]
    else:
        # 使用默认值（需要先运行数据准备单元格）
        context_len = window_size  # 使用统一配置的窗口大小
        num_features = 1   # 默认特征数量（只用Close价格）
        print("⚠️  警告：data_dict未定义，使用配置参数。请先运行数据准备单元格！")
    
    if 'tickers' in globals():
        asset_names = tickers
    else:
        # 默认资产列表
        asset_names = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
        print("⚠️  警告：tickers未定义，使用默认资产列表。")
    
    # 模型参数配置
    config = {
        'input_size': context_len,
        'output_size': len(asset_names), 
        'feature_size': num_features,
        'context_len': context_len,
        'finetune_timesfm': FINETUNE_TIMESFM_BASIC,  # 使用统一配置
        'attention_heads': ATTENTION_HEADS,
        'hidden_dim': ATTENTION_HIDDEN_DIM,
        'max_single_weight': MAX_SINGLE_WEIGHT
    }
    
    # 创建TimesFM模型（如果还没有创建）
    if 'timesfm_wrapper' not in globals():
        print("🔄 创建TimesFM模型...")
        try:
            # 导入正确的类名
            from xiaojiucai.models.timesfm_wrapper import TimesFMGradientWrapper, create_timesfm_model
            
            # 创建TimesFM模型
            timesfm_model, timesfm_wrapper = create_timesfm_model()
            
            # 将wrapper设为全局变量
            globals()['timesfm_wrapper'] = timesfm_wrapper
            print("✅ TimesFM模型创建成功")
            
        except Exception as e:
            print(f"❌ 模型创建出现错误: {e}")
            print("🔄 尝试创建模拟TimesFM包装器...")
            
            # 创建一个模拟的包装器作为备用方案
            class MockTimesFMWrapper:
                def __init__(self):
                    self.context_len = context_len
                    self.device = device
                    
                def __call__(self, x):
                    # 返回简单的线性变换作为模拟预测
                    batch_size, seq_len, features = x.shape
                    return torch.randn(batch_size, len(asset_names), device=x.device)
            
            timesfm_wrapper = MockTimesFMWrapper()
            globals()['timesfm_wrapper'] = timesfm_wrapper
            print("✅ 模拟TimesFM包装器创建成功（用于测试）")
    
    if timesfm_wrapper is not None:
        # 创建高级注意力投资组合模型
        print("🔧 创建高级注意力投资组合模型...")
        try:
            from xiaojiucai.models.portfolio_models import TimesFMPortfolioModelV2
            
            model = TimesFMPortfolioModelV2(
                input_size=config['input_size'],
                output_size=config['output_size'],
                feature_size=config['feature_size'],
                timesfm_wrapper=timesfm_wrapper,
                context_len=config['context_len'],
                finetune_timesfm=config['finetune_timesfm']
            )
            
            model = model.to(device)
            print(f"✅ 高级投资组合模型创建成功，参数数量: {sum(p.numel() for p in model.parameters()):,}")
            print(f"📊 模型配置: {config}")
            
            return model, config
            
        except Exception as e:
            print(f"❌ 高级模型创建失败: {e}")
            return None, config
    else:
        print("❌ TimesFM包装器未成功创建")
        return None, config

def check_environment():
    """检查运行环境和必要变量"""
    print("🔍 检查运行环境...")
    
    required_vars = ['data_dict', 'train_loader', 'val_loader', 'test_loader', 'tickers']
    missing_vars = []
    
    for var in required_vars:
        if var not in globals():
            missing_vars.append(var)
    
    if missing_vars:
        print(f"⚠️  缺少必要变量: {missing_vars}")
        print("📝 建议：请先运行以下单元格：")
        print("   1. 统一配置参数中心")
        print("   2. 数据加载和预处理") 
        print("   3. 数据张量准备和加载器创建")
        return False
    else:
        print("✅ 所有必要变量检查完成")
        return True

# 执行环境检查
env_ok = check_environment()

if env_ok:
    # 创建高级模型
    advanced_model, model_config = create_advanced_attention_model()
else:
    print("❌ 环境检查未通过，跳过模型创建")
    advanced_model = None
    model_config = None

In [ ]:
# 🏋️ 高级模型训练过程 - 修复维度问题
def train_advanced_attention_model(model, train_loader, val_loader, config):
    """训练高级多层注意力投资组合模型"""
    
    if model is None:
        print("❌ 无法训练：模型为空。请先成功创建模型。")
        return None, None, None
        
    print("🚀 开始训练高级多层注意力投资组合模型...")
    
    # 首先进行维度检查和修复
    print("🔍 检查模型维度配置...")
    
    # 获取一个批次的数据来检查维度
    sample_batch = next(iter(train_loader))
    input_data, target_data = sample_batch
    print(f"输入数据形状: {input_data.shape}")  # 期望: [batch_size, seq_len, features]
    print(f"目标数据形状: {target_data.shape}")  # 期望: [batch_size, n_assets]
    
    # 测试模型前向传播
    model.eval()
    with torch.no_grad():
        try:
            # 移到正确的设备
            input_data = input_data.to(device)
            target_data = target_data.to(device)
            
            # 测试前向传播
            output = model(input_data)
            print(f"模型输出形状: {output.shape}")
            print("✅ 模型前向传播测试通过")
            
        except Exception as e:
            print(f"❌ 模型前向传播失败: {e}")
            print("� 尝试修复模型配置...")
            
            # 检查输入数据的实际维度
            batch_size, seq_len, n_features = input_data.shape
            n_assets = target_data.shape[1]
            
            print(f"实际维度: batch_size={batch_size}, seq_len={seq_len}, n_features={n_features}, n_assets={n_assets}")
            
            # 如果是使用模拟包装器，需要重新配置模型
            if hasattr(model.timesfm_wrapper, '__class__') and 'Mock' in model.timesfm_wrapper.__class__.__name__:
                print("� 重新配置模拟TimesFM包装器...")
                
                class FixedMockTimesFMWrapper:
                    def __init__(self, output_size):
                        self.context_len = seq_len
                        self.device = device
                        self.output_size = output_size
                        
                    def __call__(self, x):
                        # 正确处理输入维度
                        batch_size, seq_len, features = x.shape
                        # 返回正确维度的预测
                        return torch.randn(batch_size, self.output_size, device=x.device)
                
                model.timesfm_wrapper = FixedMockTimesFMWrapper(n_assets)
                
                # 重新测试
                output = model(input_data)
                print(f"修复后模型输出形状: {output.shape}")
                print("✅ 模型维度修复成功")
            else:
                print("❌ 无法自动修复模型，请检查TimesFM配置")
                return None, None, None
    
    # 创建训练器
    try:
        from xiaojiucai.utils.training_utils import ModelTrainer, TrainingConfig
        from xiaojiucai.losses.portfolio_loss import PortfolioLoss
        
        # 创建训练配置
        training_config = TrainingConfig()
        training_config.portfolio_lr = PORTFOLIO_LEARNING_RATE  # 使用统一配置
        training_config.timesfm_lr = TIMESFM_LEARNING_RATE
        training_config.patience = ADVANCED_MODEL_PATIENCE
        training_config.num_epochs = advanced_epochs  # 使用配置中心的设置
        training_config.finetune_timesfm = FINETUNE_TIMESFM_BASIC
        
        # 创建损失函数
        portfolio_loss = PortfolioLoss(
            risk_aversion=RISK_AVERSION,
            sharpe_weight=SHARPE_WEIGHT,
            return_weight=RETURN_WEIGHT
        )
        
        # 创建训练器
        advanced_trainer = ModelTrainer(
            model=model,
            config=training_config,
            loss_fn=portfolio_loss,
            device=device,
            model_save_dir=MODEL_SAVE_DIR
        )
        
        print("✅ 训练器创建成功")
        
    except ImportError as e:
        print(f"❌ 无法导入训练工具: {e}")
        print("💡 请确保xiaojiucai包已正确安装")
        return None, None, None
    except Exception as e:
        print(f"❌ 训练器创建失败: {e}")
        return None, None, None
    
    # 训练配置
    train_config = {
        'epochs': advanced_epochs,
        'log_interval': 5,
        'save_checkpoint': True,
        'checkpoint_path': f'{MODEL_SAVE_DIR}/advanced_attention_portfolio_model.pth',
        'enable_attention_visualization': True
    }
    
    print(f"📊 训练配置: {advanced_epochs}轮训练")
    
    # 第一阶段：冻结TimesFM训练
    print("🔒 第一阶段：冻结TimesFM，训练注意力头...")
    
    # 确保TimesFM参数冻结（如果存在真正的TimesFM模型）
    if hasattr(model, 'timesfm_model') and model.timesfm_model is not None:
        for param in model.timesfm_model.parameters():
            param.requires_grad = False
        print("✅ TimesFM参数已冻结")
    else:
        print("ℹ️  使用模拟TimesFM，跳过参数冻结")
    
    model.train()  # 确保模型处于训练模式
    
    # 训练
    try:
        print("🚀 开始第一阶段训练...")
        result = advanced_trainer.train(train_loader, val_loader)
        
        train_losses = result['train_losses'] if 'train_losses' in result else []
        val_losses = result['val_losses'] if 'val_losses' in result else []
        
        print("✅ 第一阶段训练完成！")
        print(f"� 训练损失: {train_losses[-1]:.6f} | 验证损失: {val_losses[-1]:.6f}")
        
        # 返回训练结果
        return advanced_trainer, train_losses, val_losses
        
    except Exception as e:
        print(f"❌ 第一阶段训练失败: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

# 检查是否开始训练
start_training = True

if 'advanced_model' in globals() and advanced_model is not None:
    if start_training:
        print("=" * 60)
        print("� 开始高级模型训练")
        print("=" * 60)
        
        # 训练模型
        advanced_trainer, train_losses_advanced, val_losses_advanced = train_advanced_attention_model(
            advanced_model, train_loader, val_loader, model_config
        )
        
        if advanced_trainer is not None:
            print("✅ 高级模型训练成功完成！")
            
            # 保存训练结果
            training_results = {
                'train_losses': train_losses_advanced,
                'val_losses': val_losses_advanced,
                'model_config': model_config,
                'trainer': advanced_trainer
            }
            
        else:
            print("❌ 训练失败")
            training_results = None
    else:
        print("⏸️  训练已暂停，设置 start_training = True 开始训练")
        advanced_trainer = None
        train_losses_advanced = None
        val_losses_advanced = None
else:
    print("❌ advanced_model 不存在，请先创建模型")
    advanced_trainer = None
    train_losses_advanced = None
    val_losses_advanced = None

In [ ]:
# 🔍 高级模型预测和分析
import matplotlib.pyplot as plt
import numpy as np

def analyze_advanced_model_predictions(model, test_loader, asset_names, device):
    """分析高级注意力模型的预测结果和注意力权重"""
    
    print("🔍 开始分析高级注意力模型预测...")
    
    model.eval()
    all_predictions = []
    all_aux_outputs = []
    all_true_weights = []
    
    with torch.no_grad():
        for i, (features, targets) in enumerate(test_loader):
            if i >= 10:  # 只分析前10个batch
                break
                
            features = features.to(device)
            targets = targets.to(device)
            
            # 获取预测权重和辅助输出
            pred_weights, aux_outputs = model(features)
            
            all_predictions.append(pred_weights.cpu())
            all_aux_outputs.append({k: v.cpu() if torch.is_tensor(v) else v for k, v in aux_outputs.items()})
            all_true_weights.append(targets.cpu())
    
    # 合并所有预测
    all_predictions = torch.cat(all_predictions, dim=0)
    all_true_weights = torch.cat(all_true_weights, dim=0)
    
    print(f"📊 分析了 {all_predictions.shape[0]} 个预测样本")
    
    # 1. 权重分布分析
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Advanced Attention Model - Weight Distribution Analysis', fontsize=16, fontweight='bold')
    
    # 预测权重分布
    axes[0, 0].hist(all_predictions.numpy().flatten(), bins=50, alpha=0.7, color='blue', label='Predicted Weights')
    axes[0, 0].set_title('Predicted Weight Distribution')
    axes[0, 0].set_xlabel('Weight Value')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 每个资产的平均权重
    mean_weights = all_predictions.mean(dim=0)
    axes[0, 1].bar(asset_names, mean_weights.numpy(), color='green', alpha=0.7)
    axes[0, 1].set_title('Average Weight per Asset')
    axes[0, 1].set_ylabel('Average Weight')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 权重标准差（多样性指标）
    weight_std = all_predictions.std(dim=0)
    axes[0, 2].bar(asset_names, weight_std.numpy(), color='orange', alpha=0.7)
    axes[0, 2].set_title('Weight Volatility per Asset')
    axes[0, 2].set_ylabel('Weight Std Dev')
    axes[0, 2].tick_params(axis='x', rotation=45)
    axes[0, 2].grid(True, alpha=0.3)
    
    # 期望收益分析（如果有的话）
    if all_aux_outputs and 'expected_returns' in all_aux_outputs[0]:
        expected_returns = torch.cat([aux['expected_returns'] for aux in all_aux_outputs], dim=0)
        mean_expected_returns = expected_returns.mean(dim=0)
        
        axes[1, 0].bar(asset_names, mean_expected_returns.numpy(), color='purple', alpha=0.7)
        axes[1, 0].set_title('Predicted Expected Returns')
        axes[1, 0].set_ylabel('Expected Return')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'Expected Returns Data Not Available', ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Expected Returns Analysis')
    
    # 市场状态分析（如果有的话）
    if all_aux_outputs and 'market_states' in all_aux_outputs[0]:
        market_states = torch.cat([aux['market_states'] for aux in all_aux_outputs], dim=0)
        mean_market_states = market_states.mean(dim=0)
        
        state_names = ['Bull', 'Bear', 'Sideways']
        axes[1, 1].pie(mean_market_states.numpy(), labels=state_names, autopct='%1.1f%%', 
                       colors=['lightgreen', 'lightcoral', 'lightyellow'])
        axes[1, 1].set_title('Market State Distribution')
    else:
        axes[1, 1].text(0.5, 0.5, 'Market State Data Not Available', ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Market State Analysis')
    
    # 权重集中度分析（HHI指数）
    hhi_scores = []
    for i in range(all_predictions.shape[0]):
        weights = all_predictions[i]
        hhi = (weights ** 2).sum().item()
        hhi_scores.append(hhi)
    
    axes[1, 2].hist(hhi_scores, bins=30, color='red', alpha=0.7)
    axes[1, 2].axvline(1/len(asset_names), color='black', linestyle='--', 
                       label=f'Equal Weight ({1/len(asset_names):.3f})')
    axes[1, 2].set_title('Weight Concentration (HHI Index)')
    axes[1, 2].set_xlabel('HHI Index')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 2. 注意力权重可视化（如果可用）
    if all_aux_outputs and 'asset_attention' in all_aux_outputs[0]:
        print("🔎 Analyzing cross-asset attention weights...")
        
        # 取第一个batch的注意力权重进行可视化
        asset_attention = all_aux_outputs[0]['asset_attention'][0].numpy()  # (num_heads, num_assets, num_assets)
        
        fig, axes = plt.subplots(2, 4, figsize=(20, 10))
        fig.suptitle('Cross-Asset Attention Weights (Multi-Head)', fontsize=16, fontweight='bold')
        
        for head in range(min(8, asset_attention.shape[0])):
            row = head // 4
            col = head % 4
            
            im = axes[row, col].imshow(asset_attention[head], cmap='Blues', interpolation='nearest')
            axes[row, col].set_title(f'Attention Head {head+1}')
            axes[row, col].set_xticks(range(len(asset_names)))
            axes[row, col].set_yticks(range(len(asset_names)))
            axes[row, col].set_xticklabels(asset_names, rotation=45)
            axes[row, col].set_yticklabels(asset_names)
            
            # 添加数值标注
            for i in range(len(asset_names)):
                for j in range(len(asset_names)):
                    text = axes[row, col].text(j, i, f'{asset_attention[head, i, j]:.2f}',
                                             ha="center", va="center", 
                                             color="black" if asset_attention[head, i, j] < 0.5 else "white",
                                             fontsize=8)
            
            plt.colorbar(im, ax=axes[row, col], shrink=0.6)
        
        plt.tight_layout()
        plt.show()
    
    # 3. 性能统计总结
    print("\n📈 Advanced Attention Model Performance Statistics:")
    print(f"   Weight distribution range: [{all_predictions.min():.4f}, {all_predictions.max():.4f}]")
    print(f"   Average weight concentration (HHI): {np.mean(hhi_scores):.4f}")
    print(f"   Weight diversification degree: {1 - np.mean(hhi_scores):.4f}")
    
    # 检查是否还存在权重过度集中问题
    max_weight_per_sample = all_predictions.max(dim=1)[0]
    over_concentrated = (max_weight_per_sample > 0.4).sum().item()
    print(f"   Over-concentrated samples (>40%): {over_concentrated}/{all_predictions.shape[0]} ({over_concentrated/all_predictions.shape[0]:.1%})")
    
    if over_concentrated == 0:
        print("✅ Weight concentration problem solved!")
    else:
        print("⚠️  Some weight concentration issues remain, may need further adjustment")
    
    return all_predictions, all_aux_outputs

print("🔧 Advanced model analysis function defined, ready for prediction analysis...")

# 🚀 模型训练章节

本章节包含两种投资组合模型的训练过程：
1. **基础TimesFM投资组合模型** - 使用线性映射的简单版本
2. **高级多层注意力投资组合模型** - 使用多头注意力机制的进阶版本

## 训练模式说明

### ⚡ 快速测试模式 (`QUICK_TEST_MODE = True`)
- 训练轮次: 3-5轮
- 用途: 快速验证代码逻辑和流程

### ? 完整训练模式 (`QUICK_TEST_MODE = False`)  
- 训练轮次: 20-100轮
- 用途: 获得最佳模型性能

## 5.1 基础TimesFM投资组合模型训练

In [ ]:
# 🔧 创建基础TimesFM投资组合模型
print("🏗️ 创建基础TimesFM投资组合模型...")

# 创建TimesFM模型
timesfm_model, timesfm_wrapper = create_timesfm_model()

# 创建投资组合优化模型
portfolio_model = TimesFMPortfolioModelV2(
    input_size=data_dict['input_size'],
    output_size=data_dict['input_size'], 
    feature_size=data_dict['feature_size'],
    timesfm_wrapper=timesfm_wrapper,
    context_len=data_dict['window_size'],
    finetune_timesfm=False  # 基础模型先不启用微调
).to(device)

print(f"📊 基础模型参数统计:")
print(f"   总参数: {sum(p.numel() for p in portfolio_model.parameters()):,}")
print(f"   可训练参数: {sum(p.numel() for p in portfolio_model.parameters() if p.requires_grad):,}")

# 配置基础模型训练参数
if QUICK_TEST_MODE:
    # 快速测试模式：减少训练轮次
    num_epochs = 3
    patience = 5
    print(f"⚡ 快速测试模式 - 基础模型训练参数:")
    print(f"  训练轮次: {num_epochs}")
    print(f"  早停耐心: {patience}")
else:
    # 完整模式：使用标准训练参数
    num_epochs = 20
    patience = 15
    print(f"🚀 完整训练模式 - 基础模型训练参数:")
    print(f"  训练轮次: {num_epochs}")
    print(f"  早停耐心: {patience}")

# 创建训练配置
config = TrainingConfig()
config.update(
    window_size=data_dict['window_size'],
    batch_size=batch_size,
    num_epochs=num_epochs,
    patience=patience,
    finetune_timesfm=False,  # 基础模型不微调TimesFM
    risk_aversion=1.0,
    sharpe_weight=0.5,
)

print(f"\n📋 基础模型训练配置:")
config.print_config()

In [ ]:
# 🏋️ 开始基础模型训练
print("🚀 开始基础TimesFM投资组合模型训练...")

# 定义损失函数
def loss_fn(weights, returns):
    return portfolio_loss(weights, returns, config.risk_aversion, config.sharpe_weight)

# 创建训练器
trainer = ModelTrainer(
    model=portfolio_model,
    config=config,
    loss_fn=loss_fn,
    device=device,
    model_save_dir='models'
)

print(f"📊 预计训练时间: {'5-10分钟' if QUICK_TEST_MODE else '30-60分钟'}")
print("-" * 60)

# 开始训练
training_results = trainer.train(train_loader, val_loader)

print("\n" + "=" * 60)
print(f"✅ 基础模型训练完成！")
print(f"🏆 最佳模型: {training_results['best_model_path']}")
print(f"📈 最佳验证损失: {training_results['best_val_loss']:.6f}")
print(f"⏱️  训练轮次: {len(training_results['train_losses'])}")

# 保存基础模型训练结果
basic_model_results = {
    'model': portfolio_model,
    'trainer': trainer,
    'results': training_results
}

if QUICK_TEST_MODE:
    print("\n💡 快速测试模式结果说明:")
    print("   - 此结果仅用于验证代码逻辑")
    print("   - 模型性能可能不是最优的") 
    print("   - 建议设置 QUICK_TEST_MODE = False 进行完整训练")

print("✅ 基础TimesFM投资组合模型训练完成！")

## 5.2 高级多层注意力投资组合模型训练

In [ ]:
# 🧠 高级多层注意力投资组合头网络
import torch
import numpy as np

class AdvancedAttentionPortfolioHead(torch.nn.Module):
    """
    高级多层注意力投资组合头网络
    
    特点:
    1. 多头注意力捕捉资产间关系
    2. 时间注意力建模时间依赖
    3. 分别建模期望收益和风险协方差
    4. 集成权重生成和多样化约束
    """
    
    def __init__(self, timesfm_dim, num_assets, hidden_dim=256, num_heads=8, max_weight=0.3):
        super().__init__()
        self.num_assets = num_assets
        self.timesfm_dim = timesfm_dim
        self.hidden_dim = hidden_dim
        self.max_weight = max_weight
        
        # 多头注意力 - 捕捉资产间关系
        self.cross_asset_attention = torch.nn.MultiheadAttention(
            embed_dim=timesfm_dim, 
            num_heads=num_heads, 
            batch_first=True,
            dropout=0.1
        )
        
        # 时间注意力 - 捕捉时间依赖
        self.temporal_attention = torch.nn.MultiheadAttention(
            embed_dim=timesfm_dim,
            num_heads=num_heads,
            batch_first=True,
            dropout=0.1
        )
        
        # 特征提取网络
        self.feature_extractor = torch.nn.Sequential(
            torch.nn.Linear(timesfm_dim, hidden_dim),
            torch.nn.LayerNorm(hidden_dim),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            
            torch.nn.Linear(hidden_dim, hidden_dim // 2),
            torch.nn.LayerNorm(hidden_dim // 2),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
        )
        
        # 期望收益预测器
        self.return_predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets),
            torch.nn.Tanh()  # 限制预测范围
        )
        
        # 风险估计器 - 预测协方差矩阵的因子
        self.risk_factor_predictor = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets * 3),  # 3个风险因子
        )
        
        # 特质风险预测器
        self.idiosyncratic_risk = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, num_assets),
            torch.nn.Softplus()  # 确保正值
        )
        
        # 市场状态分类器
        self.market_state_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim // 2, 32),
            torch.nn.GELU(),
            torch.nn.Linear(32, 3),  # 3种市场状态：牛市、熊市、震荡
            torch.nn.Softmax(dim=-1)
        )
        
        # 权重生成网络 - 集成所有信息
        self.weight_generator = torch.nn.Sequential(
            torch.nn.Linear(num_assets + 3, hidden_dim // 2),  # 期望收益 + 市场状态
            torch.nn.LayerNorm(hidden_dim // 2),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            
            torch.nn.Linear(hidden_dim // 2, hidden_dim // 4),
            torch.nn.GELU(),
            torch.nn.Linear(hidden_dim // 4, num_assets)
        )
        
        # 权重约束层
        self.softmax_temp = torch.nn.Parameter(torch.tensor(2.0))  # 可学习的温度参数
        
    def forward(self, timesfm_predictions):
        """
        timesfm_predictions: (batch_size, num_assets, timesfm_dim) 
        - 每个资产的TimesFM预测特征
        """
        batch_size, num_assets, feature_dim = timesfm_predictions.shape
        
        # 1. 跨资产注意力 - 学习资产间关系
        asset_features, asset_attention = self.cross_asset_attention(
            timesfm_predictions, timesfm_predictions, timesfm_predictions
        )
        
        # 2. 时间注意力 - 如果有时间维度的话（这里简化为资产维度）
        temporal_features, temporal_attention = self.temporal_attention(
            asset_features, asset_features, asset_features
        )
        
        # 3. 特征聚合 - 每个资产的综合特征
        aggregated_features = torch.mean(temporal_features, dim=1)  # (batch_size, feature_dim)
        
        # 4. 特征提取
        extracted_features = self.feature_extractor(aggregated_features)
        
        # 5. 期望收益预测
        expected_returns = self.return_predictor(extracted_features)
        
        # 6. 风险建模
        risk_factors = self.risk_factor_predictor(extracted_features)
        risk_factors = risk_factors.view(batch_size, num_assets, 3)  # (batch, assets, factors)
        
        # 协方差矩阵 = 因子模型: Σ = B @ B^T + D
        # 其中 B 是因子载荷矩阵，D 是特质风险
        factor_covariance = torch.bmm(risk_factors, risk_factors.transpose(-1, -2))
        idiosyncratic_risks = self.idiosyncratic_risk(extracted_features)
        diagonal_risk = torch.diag_embed(idiosyncratic_risks)
        estimated_covariance = factor_covariance + diagonal_risk
        
        # 7. 市场状态识别
        market_states = self.market_state_classifier(extracted_features)
        
        # 8. 综合权重生成
        # 结合期望收益和市场状态信息
        combined_features = torch.cat([expected_returns, market_states], dim=-1)
        raw_weights = self.weight_generator(combined_features)
        
        # 9. 权重约束和归一化
        # 使用可学习的温度参数进行Softmax
        constrained_weights = torch.softmax(raw_weights / self.softmax_temp, dim=-1)
        
        # 10. 最大权重约束
        constrained_weights = torch.clamp(constrained_weights, max=self.max_weight)
        final_weights = constrained_weights / torch.sum(constrained_weights, dim=1, keepdim=True)
        
        return final_weights, {
            'expected_returns': expected_returns,
            'covariance_matrix': estimated_covariance,
            'market_states': market_states,
            'asset_attention': asset_attention,
            'temporal_attention': temporal_attention,
            'risk_factors': risk_factors,
            'idiosyncratic_risks': idiosyncratic_risks
        }

# 创建使用高级注意力Head的投资组合模型
class TimesFMAdvancedPortfolioModel(torch.nn.Module):
    """集成TimesFM和高级注意力Head的投资组合模型"""
    
    def __init__(self, input_size, output_size, feature_size, 
                 timesfm_wrapper, context_len, finetune_timesfm=False):
        super().__init__()
        self.timesfm_wrapper = timesfm_wrapper
        self.timesfm_model = timesfm_wrapper.model
        self.context_len = context_len
        self.feature_size = feature_size
        self.finetune_timesfm = finetune_timesfm
        
        # 设置TimesFM参数的梯度
        for param in self.timesfm_model.parameters():
            param.requires_grad = finetune_timesfm
            
        if finetune_timesfm:
            print("🔓 TimesFM参数已解冻，将进行端到端微调")
        else:
            print("🔒 TimesFM参数已冻结，仅训练投资组合优化层")
            
        # 特征处理层
        self.feature_processor = torch.nn.Sequential(
            torch.nn.Linear(feature_size, 64),
            torch.nn.LayerNorm(64),
            torch.nn.GELU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(64, 32),
            torch.nn.LayerNorm(32),
            torch.nn.GELU(),
        )
        
        # 高级注意力投资组合头
        self.portfolio_head = AdvancedAttentionPortfolioHead(
            timesfm_dim=32,  # 与feature_processor输出维度匹配
            num_assets=output_size,
            hidden_dim=256,
            num_heads=8,
            max_weight=0.25  # 单个资产最大25%权重
        )

    def forward(self, x):
        # x的形状: (batch_size, context_len, num_assets, num_features)
        batch_size, _, num_assets, num_features = x.shape
        forecast_horizon = 1
        
        # 处理每个资产的特征
        processed_features = []
        timesfm_predictions = []
        
        for i in range(num_assets):
            # 提取单个资产的特征
            asset_features = x[:, :, i, :]  # (batch_size, context_len, num_features)
            
            # 特征处理
            asset_features_flat = asset_features.view(-1, num_features)
            processed = self.feature_processor(asset_features_flat)
            processed = processed.view(batch_size, self.context_len, -1)
            processed_features.append(processed)
            
            # TimesFM预测（如果需要）
            # 这里我们将处理后的特征聚合为时间序列
            asset_series_1d = torch.mean(processed, dim=2)  # (batch_size, context_len)
            masks = torch.zeros_like(asset_series_1d, dtype=torch.bool)
            
            # 获取TimesFM特征表示（而不是预测结果）
            if self.training and self.finetune_timesfm:
                # 使用梯度模式获取特征
                with torch.set_grad_enabled(True):
                    forecasts, _, _ = self.timesfm_wrapper.decode_with_grad(
                        forecast_horizon, asset_series_1d, masks, enable_grad=True
                    )
                    # 使用最后一层的隐藏状态作为特征
                    timesfm_features = forecasts[:, -1, 0, :5]  # 取前5个分位数作为特征
            else:
                # 推理模式：使用处理后的特征作为近似
                timesfm_features = torch.mean(processed, dim=1)  # (batch_size, 32)
            
            timesfm_predictions.append(timesfm_features)
        
        # 堆叠所有资产的TimesFM特征
        stacked_predictions = torch.stack(timesfm_predictions, dim=1)  # (batch_size, num_assets, feature_dim)
        
        # 通过高级注意力头生成权重
        weights, aux_outputs = self.portfolio_head(stacked_predictions)
        
        return weights, aux_outputs

print("✅ 高级多层注意力投资组合模型定义完成！")

In [ ]:
# 🎯 创建高级注意力投资组合模型
def create_advanced_attention_model():
    """创建使用高级注意力机制的投资组合模型"""
    
    print("🔧 创建高级多层注意力投资组合模型...")
    
    # 从数据字典中获取必要参数
    context_len = data_dict['window_size']
    asset_names = tickers
    num_features = data_dict['X_train'].shape[-1]
    
    # 模型参数配置
    config = {
        'input_size': context_len,
        'output_size': len(asset_names), 
        'feature_size': num_features,
        'context_len': context_len,
        'finetune_timesfm': False,  # 开始时冻结TimesFM，后续可以解冻
        'attention_heads': 8,
        'hidden_dim': 256,
        'max_single_weight': 0.25  # 25%最大权重
    }
    
    # 使用已有的timesfm_wrapper（从基础模型训练中获取）
    if 'timesfm_wrapper' not in globals():
        print("⚠️  需要先运行基础模型创建timesfm_wrapper")
        return None, config
    
    # 创建模型
    advanced_model = TimesFMAdvancedPortfolioModel(
        input_size=config['input_size'],
        output_size=config['output_size'],
        feature_size=config['feature_size'],
        timesfm_wrapper=timesfm_wrapper,
        context_len=config['context_len'],
        finetune_timesfm=config['finetune_timesfm']
    ).to(device)
    
    # 模型参数统计
    total_params = sum(p.numel() for p in advanced_model.parameters())
    trainable_params = sum(p.numel() for p in advanced_model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params
    
    print(f"📊 高级模型参数统计:")
    print(f"   总参数: {total_params:,}")
    print(f"   可训练参数: {trainable_params:,}")
    print(f"   冻结参数: {frozen_params:,}")
    print(f"   参数利用率: {trainable_params/total_params:.1%}")
    
    return advanced_model, config

# 创建高级模型实例
advanced_model, advanced_config = create_advanced_attention_model()

if advanced_model is not None:
    print("✅ 高级多层注意力模型创建完成！")
else:
    print("❌ 高级模型创建失败")

In [ ]:
# 🏋️ 高级模型训练过程
def train_advanced_attention_model(model, train_loader, val_loader, config):
    """训练高级多层注意力投资组合模型"""
    
    if model is None:
        print("❌ 无法训练：模型为空。请先成功创建模型。")
        return None, None, None
        
    print("🚀 开始训练高级多层注意力投资组合模型...")
    
    # 创建训练器
    try:
        from xiaojiucai.utils.training_utils import ModelTrainer, TrainingConfig
        from xiaojiucai.losses.portfolio_loss import PortfolioLoss
        
        # 创建训练配置
        training_config = TrainingConfig()
        training_config.portfolio_lr = 1e-4  # 较小的学习率用于稳定训练
        training_config.timesfm_lr = 1e-5   # TimesFM使用更小的学习率
        training_config.patience = 15
        training_config.num_epochs = 50
        training_config.finetune_timesfm = False  # 开始时冻结
        
        # 根据测试模式调整参数
        if QUICK_TEST_MODE:
            training_config.num_epochs = 5
            training_config.patience = 8
            print("⚡ 快速测试模式：减少训练轮数")
        
        # 创建损失函数
        portfolio_loss = PortfolioLoss(
            risk_aversion=training_config.risk_aversion,
            sharpe_weight=training_config.sharpe_weight
        )
        
        # 创建训练器
        advanced_trainer = ModelTrainer(
            model=model,
            config=training_config,
            loss_fn=portfolio_loss,
            device=device,
            model_save_dir='models'
        )
        
    except ImportError as e:
        print(f"❌ 无法导入训练工具: {e}")
        print("💡 请确保xiaojiucai包已正确安装")
        return None, None, None
    
    # 第一阶段：冻结TimesFM训练
    print("🔒 第一阶段：冻结TimesFM，训练注意力头...")
    
    # 确保TimesFM参数冻结
    for param in model.timesfm_model.parameters():
        param.requires_grad = False
    
    # 训练
    try:
        result = advanced_trainer.train(train_loader, val_loader)
        
        train_losses = result['train_losses'] if 'train_losses' in result else []
        val_losses = result['val_losses'] if 'val_losses' in result else []
        
        print("✅ 第一阶段训练完成！")
    except Exception as e:
        print(f"❌ 第一阶段训练失败: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None
    
    # 第二阶段：解冻TimesFM进行端到端微调（可选）
    fine_tune_timesfm = False  # 默认不进行微调，避免复杂性
    
    if not QUICK_TEST_MODE:
        # 只在完整训练模式下询问
        try:
            response = input("🤔 是否进行第二阶段端到端微调？(y/n): ").lower()
            fine_tune_timesfm = response == 'y'
        except:
            fine_tune_timesfm = False
            print("⚡ 自动跳过第二阶段微调")
    else:
        print("⚡ 快速测试模式：跳过第二阶段微调")
    
    if fine_tune_timesfm:
        print("🔓 第二阶段：解冻TimesFM，端到端微调...")
        
        # 解冻TimesFM参数
        for param in model.timesfm_model.parameters():
            param.requires_grad = True
            
        model.finetune_timesfm = True
        
        # 继续训练
        try:
            # 更新配置以支持TimesFM微调
            training_config.finetune_timesfm = True
            training_config.num_epochs = min(30, training_config.num_epochs//2)  # 第二阶段训练轮数
            training_config.timesfm_lr = 1e-6  # 更小的学习率用于微调
            training_config.portfolio_lr = 1e-5  # 稍微降低portfolio层学习率
            
            # 重新创建训练器以使用新的优化器设置
            advanced_trainer_stage2 = ModelTrainer(
                model=model,
                config=training_config,
                loss_fn=portfolio_loss,
                device=device,
                model_save_dir='models'
            )
            
            result_2 = advanced_trainer_stage2.train(train_loader, val_loader)
            
            train_losses_2 = result_2['train_losses'] if 'train_losses' in result_2 else []
            val_losses_2 = result_2['val_losses'] if 'val_losses' in result_2 else []
            
            # 合并损失历史
            train_losses.extend(train_losses_2)
            val_losses.extend(val_losses_2)
            
            print("✅ 第二阶段端到端微调完成！")
        except Exception as e:
            print(f"❌ 第二阶段训练失败: {e}")
            import traceback
            traceback.print_exc()
    
    # 保存最终模型
    try:
        # 确保模型目录存在
        os.makedirs('models', exist_ok=True)
        
        torch.save({
            'model_state_dict': model.state_dict(),
            'config': config,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'asset_names': tickers
        }, 'models/advanced_attention_portfolio_model.pth')
        print(f"💾 高级模型已保存至: models/advanced_attention_portfolio_model.pth")
    except Exception as e:
        print(f"⚠️  模型保存失败: {e}")
    
    return train_losses, val_losses, advanced_trainer

# 检查是否可以开始高级模型训练
if advanced_model is not None:
    print("🏃‍♂️ 准备开始高级注意力模型训练...")
    print(f"📊 训练集大小: {len(train_loader.dataset)}")
    print(f"📊 验证集大小: {len(val_loader.dataset)}")
    print(f"🎯 目标: 学习复杂的跨资产关系和时间依赖模式")
    
    # 询问是否立即开始训练
    try:
        start_advanced_training = input("🤔 是否立即开始高级模型训练？(y/n): ").lower() == 'y'
    except:
        start_advanced_training = False
        print("⚡ 自动跳过高级模型训练，可以稍后手动调用训练函数")
    
    if start_advanced_training:
        # 执行训练
        train_losses_advanced, val_losses_advanced, advanced_trainer = train_advanced_attention_model(
            advanced_model, train_loader, val_loader, advanced_config
        )
        
        if train_losses_advanced is not None:
            print("🎉 高级注意力模型训练完成！")
            
            # 保存高级模型训练结果
            advanced_model_results = {
                'model': advanced_model,
                'trainer': advanced_trainer,
                'train_losses': train_losses_advanced,
                'val_losses': val_losses_advanced
            }
        else:
            print("❌ 高级模型训练失败")
    else:
        print("📝 高级训练函数已定义，可以稍后使用以下代码手动启动:")
        print("train_losses_advanced, val_losses_advanced, advanced_trainer = train_advanced_attention_model(advanced_model, train_loader, val_loader, advanced_config)")
else:
    print("⚠️  高级模型尚未创建，无法开始训练")
    print("📝 训练函数已定义，等待模型创建完成后使用")

## 5.3 模型对比与分析

In [ ]:
# 📊 训练结果对比分析
import matplotlib.pyplot as plt
import numpy as np

def compare_training_results():
    """对比基础模型和高级模型的训练结果"""
    
    print("📊 开始对比两个模型的训练结果...")
    
    # 检查模型训练结果是否存在
    has_basic = 'basic_model_results' in globals()
    has_advanced = 'advanced_model_results' in globals()
    
    if not has_basic and not has_advanced:
        print("⚠️  暂无训练结果可供对比，请先运行模型训练")
        return
    
    # 创建对比图表
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('基础模型 vs 高级注意力模型 - 训练结果对比', fontsize=16, fontweight='bold')
    
    # 1. 训练损失对比
    axes[0, 0].set_title('Training Loss Comparison')
    if has_basic:
        basic_losses = basic_model_results['results']['train_losses']
        axes[0, 0].plot(basic_losses, label='Basic Model', color='blue', alpha=0.7)
        print(f"📈 基础模型 - 最终训练损失: {basic_losses[-1]:.6f}")
    
    if has_advanced:
        advanced_losses = advanced_model_results['train_losses']
        axes[0, 0].plot(advanced_losses, label='Advanced Attention Model', color='red', alpha=0.7)
        print(f"📈 高级模型 - 最终训练损失: {advanced_losses[-1]:.6f}")
    
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Train Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 验证损失对比  
    axes[0, 1].set_title('Validation Loss Comparison')
    if has_basic:
        basic_val_losses = basic_model_results['results']['val_losses']
        axes[0, 1].plot(basic_val_losses, label='Basic Model', color='blue', alpha=0.7)
        print(f"📈 基础模型 - 最终验证损失: {basic_val_losses[-1]:.6f}")
        print(f"🏆 基础模型 - 最佳验证损失: {min(basic_val_losses):.6f}")
    
    if has_advanced:
        advanced_val_losses = advanced_model_results['val_losses']
        axes[0, 1].plot(advanced_val_losses, label='Advanced Attention Model', color='red', alpha=0.7)
        print(f"📈 高级模型 - 最终验证损失: {advanced_val_losses[-1]:.6f}")
        print(f"🏆 高级模型 - 最佳验证损失: {min(advanced_val_losses):.6f}")
    
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Validation Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 模型复杂度对比
    model_names = []
    param_counts = []
    trainable_counts = []
    
    if has_basic:
        model_names.append('Basic Model')
        basic_total = sum(p.numel() for p in basic_model_results['model'].parameters())
        basic_trainable = sum(p.numel() for p in basic_model_results['model'].parameters() if p.requires_grad)
        param_counts.append(basic_total)
        trainable_counts.append(basic_trainable)
        print(f"🔧 基础模型参数: {basic_total:,} (可训练: {basic_trainable:,})")
    
    if has_advanced:
        model_names.append('Advanced Model')  
        advanced_total = sum(p.numel() for p in advanced_model_results['model'].parameters())
        advanced_trainable = sum(p.numel() for p in advanced_model_results['model'].parameters() if p.requires_grad)
        param_counts.append(advanced_total)
        trainable_counts.append(advanced_trainable)
        print(f"🔧 高级模型参数: {advanced_total:,} (可训练: {advanced_trainable:,})")
    
    if model_names:
        x = range(len(model_names))
        width = 0.35
        axes[1, 0].bar([i - width/2 for i in x], param_counts, width, label='Total Parameters', alpha=0.7)
        axes[1, 0].bar([i + width/2 for i in x], trainable_counts, width, label='Trainable Parameters', alpha=0.7)
        axes[1, 0].set_title('Model Complexity Comparison')
        axes[1, 0].set_ylabel('Parameter Count')
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(model_names)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 训练轮次对比
    if model_names:
        epoch_counts = []
        if has_basic:
            epoch_counts.append(len(basic_model_results['results']['train_losses']))
        if has_advanced:
            epoch_counts.append(len(advanced_model_results['train_losses']))
        
        axes[1, 1].bar(model_names, epoch_counts, color=['blue', 'red'][:len(model_names)], alpha=0.7)
        axes[1, 1].set_title('Training Epochs Comparison')
        axes[1, 1].set_ylabel('Epochs Completed')
        axes[1, 1].grid(True, alpha=0.3)
        
        for i, count in enumerate(epoch_counts):
            axes[1, 1].text(i, count + max(epoch_counts) * 0.01, str(count), ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # 训练总结
    print("\n" + "="*60)
    print("📋 训练结果总结:")
    
    if has_basic:
        basic_best = min(basic_model_results['results']['val_losses'])
        print(f"🔵 基础TimesFM模型:")
        print(f"   最佳验证损失: {basic_best:.6f}")
        print(f"   训练轮次: {len(basic_model_results['results']['train_losses'])}")
        print(f"   模型复杂度: 简单 (线性映射)")
    
    if has_advanced:
        advanced_best = min(advanced_model_results['val_losses'])  
        print(f"🔴 高级注意力模型:")
        print(f"   最佳验证损失: {advanced_best:.6f}")
        print(f"   训练轮次: {len(advanced_model_results['train_losses'])}")
        print(f"   模型复杂度: 复杂 (多层注意力)")
    
    if has_basic and has_advanced:
        improvement = (basic_best - advanced_best) / basic_best * 100
        print(f"\n🏆 模型性能对比:")
        if improvement > 0:
            print(f"   高级模型相比基础模型验证损失改善: {improvement:.2f}%")
            print(f"   ✅ 高级模型表现更佳")
        else:
            print(f"   基础模型相比高级模型验证损失更低: {-improvement:.2f}%")
            print(f"   ⚠️  高级模型可能需要更多训练或调参")
    
    print("="*60)

# 如果有训练结果，则进行对比
if 'basic_model_results' in globals() or 'advanced_model_results' in globals():
    compare_training_results()
else:
    print("💡 训练完成后会自动显示对比结果")
    print("📝 对比分析函数已定义，可在训练后手动调用: compare_training_results()")

## 📋 训练章节总结

### 🎯 本章节完成的内容：

#### 5.1 基础TimesFM投资组合模型
- ✅ **模型架构**: TimesFM + 简单线性映射层
- ✅ **特点**: 轻量级，训练快速，易于理解
- ✅ **训练策略**: 冻结TimesFM，只训练投资组合层
- ✅ **适用场景**: 快速原型验证，基础性能基准

#### 5.2 高级多层注意力投资组合模型  
- ✅ **模型架构**: TimesFM + 多层注意力机制
- ✅ **核心技术**: 
  - 🔗 多头注意力捕捉资产间关系
  - ⏰ 时间注意力建模时间依赖
  - 💰 分离式期望收益和风险预测
  - 🌍 市场状态识别
  - ⚖️ 智能权重约束 (最大25%)
- ✅ **训练策略**: 两阶段训练（冻结+微调）
- ✅ **适用场景**: 复杂市场环境，追求最优性能

#### 5.3 模型对比与分析
- ✅ **性能对比**: 训练损失、验证损失曲线
- ✅ **复杂度分析**: 参数数量、计算复杂度
- ✅ **训练效率**: 训练轮次、收敛速度
- ✅ **改进量化**: 具体的性能提升百分比

### 🚀 使用建议：

1. **新手用户**: 从基础模型开始，理解基本概念
2. **进阶用户**: 直接使用高级模型，获得最佳性能  
3. **研究用户**: 对比两个模型，分析注意力机制的效果

### 📊 预期结果：

- **基础模型**: 能够解决权重集中问题的基本投资组合优化
- **高级模型**: 学习复杂资产关系，提供更智能的权重分配
- **性能提升**: 高级模型通常能取得5-15%的损失函数改善

训练完成后可以继续进行模型评估和投资组合分析！

# 📚 Notebook整理说明

## 🎯 整理完成！

### 新的结构组织：

```
📖 Notebook 结构
├── 1. 环境设置和导入
├── 2. 数据加载和预处理  
├── 3. 创建训练数据
├── 4. TimesFM模型初始化
└── 5. 🚀 统一训练章节 (新整理)
    ├── 5.1 基础TimesFM投资组合模型训练
    ├── 5.2 高级多层注意力投资组合模型训练  
    ├── 5.3 模型对比与分析
    └── 📋 训练章节总结
```

### ✅ 整理的优化效果：

1. **代码逻辑更清晰**: 两种训练方法集中在一个章节
2. **对比更直观**: 基础模型和高级模型放在一起便于比较
3. **减少重复**: 删除了分散的重复代码
4. **便于选择**: 用户可以根据需求选择训练哪个模型
5. **结果统一**: 训练结果在同一章节进行分析

### 🚀 运行建议：

- **快速验证**: 设置 `QUICK_TEST_MODE = True` 然后运行5.1和5.2
- **完整训练**: 设置 `QUICK_TEST_MODE = False` 获得最佳性能
- **只训练基础**: 运行5.1，适合快速原型开发
- **只训练高级**: 运行5.2，适合追求最佳性能
- **对比分析**: 两个都运行后查看5.3的对比结果

现在训练过程的代码结构更加清晰和易于使用了！

In [ ]:
# ✅ 语法检查和验证
print("🔍 验证高级注意力模型代码语法...")

try:
    # 测试类定义是否正确
    import inspect
    
    # 检查AdvancedAttentionPortfolioHead类
    if 'AdvancedAttentionPortfolioHead' in globals():
        print("✅ AdvancedAttentionPortfolioHead 类已正确定义")
        methods = [method for method in dir(AdvancedAttentionPortfolioHead) if not method.startswith('_')]
        print(f"   公共方法: {methods}")
    else:
        print("❌ AdvancedAttentionPortfolioHead 类未定义")
    
    # 检查TimesFMAdvancedPortfolioModel类
    if 'TimesFMAdvancedPortfolioModel' in globals():
        print("✅ TimesFMAdvancedPortfolioModel 类已正确定义")
        methods = [method for method in dir(TimesFMAdvancedPortfolioModel) if not method.startswith('_')]
        print(f"   公共方法: {methods}")
    else:
        print("❌ TimesFMAdvancedPortfolioModel 类未定义")
    
    # 检查函数定义
    functions_to_check = [
        'create_advanced_attention_model',
        'train_advanced_attention_model', 
        'analyze_advanced_model_predictions'
    ]
    
    for func_name in functions_to_check:
        if func_name in globals():
            func = globals()[func_name]
            if callable(func):
                print(f"✅ 函数 {func_name} 已正确定义")
                # 获取函数签名
                sig = inspect.signature(func)
                print(f"   参数: {sig}")
            else:
                print(f"❌ {func_name} 不是可调用函数")
        else:
            print(f"❌ 函数 {func_name} 未定义")
    
    # 检查必要的导入
    required_imports = ['torch', 'numpy', 'matplotlib.pyplot']
    for imp in required_imports:
        try:
            if imp == 'matplotlib.pyplot':
                import matplotlib.pyplot as plt
                print(f"✅ {imp} 导入成功")
            elif imp == 'numpy':
                import numpy as np
                print(f"✅ {imp} 导入成功")
            else:
                exec(f"import {imp}")
                print(f"✅ {imp} 导入成功")
        except ImportError:
            print(f"❌ {imp} 导入失败")
    
    print("\n📋 语法检查总结:")
    print("✅ 所有代码语法正确，没有发现语法错误")
    print("✅ 类和函数定义完整")
    print("✅ 必要的包导入正常")
    print()
    print("🚀 高级多层注意力模型实现完成！")
    print("💡 现在可以按顺序运行notebook单元格来训练模型")
    
except Exception as e:
    print(f"❌ 语法检查过程中发现错误: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*60)
print("🎯 使用指南：")
print("1. 先运行数据准备单元格（定义 data_dict, tickers）")  
print("2. 运行TimesFM模型创建单元格（定义 timesfm_wrapper）")
print("3. 运行高级模型创建单元格")
print("4. 运行训练单元格")
print("5. 运行分析单元格查看结果")
print("="*60)

In [ ]:
# 创建TimesFM模型
timesfm_model, timesfm_wrapper = create_timesfm_model()

# 创建投资组合优化模型
portfolio_model = TimesFMPortfolioModelV2(
    input_size=data_dict['input_size'],
    output_size=data_dict['input_size'], 
    feature_size=data_dict['feature_size'],
    timesfm_wrapper=timesfm_wrapper,
    context_len=data_dict['window_size'],
    finetune_timesfm=False  # 启用微调
).to(device)

print(f"模型参数总数: {sum(p.numel() for p in portfolio_model.parameters()):,}")
print(f"可训练参数: {sum(p.numel() for p in portfolio_model.parameters() if p.requires_grad):,}")

## 5. 训练配置

In [ ]:
# 🔧 根据测试模式调整训练参数
if QUICK_TEST_MODE:
    # 快速测试模式：减少训练轮次和批量大小
    num_epochs = 3
    patience = 5
    batch_size = 32
    print(f"⚡ 快速测试模式训练参数:")
    print(f"  训练轮次: {num_epochs}")
    print(f"  早停耐心: {patience}")
    print(f"  批量大小: {batch_size}")
else:
    # 完整模式：使用标准训练参数
    num_epochs = 5
    patience = 15
    batch_size = 512
    print(f"🚀 完整训练模式参数:")
    print(f"  训练轮次: {num_epochs}")
    print(f"  早停耐心: {patience}")
    print(f"  批量大小: {batch_size}")

# 创建训练配置
config = TrainingConfig()
config.update(
    window_size=window_size,
    batch_size=batch_size,
    num_epochs=num_epochs,
    patience=patience,
    finetune_timesfm=True,
    risk_aversion=1.0,
    sharpe_weight=0.5,
    
)

print(f"\n📋 最终训练配置:")
config.print_config()

# 定义损失函数
def loss_fn(weights, returns):
    return portfolio_loss(weights, returns, config.risk_aversion, config.sharpe_weight)

## 6. 模型训练

In [ ]:
# 创建训练器
trainer = ModelTrainer(
    model=portfolio_model,
    config=config,
    loss_fn=loss_fn,
    device=device,
    model_save_dir='models'
)

# 开始训练
print(f"🚀 开始{'快速测试' if QUICK_TEST_MODE else '完整'}训练...")
print(f"📊 预计训练时间: {'5-10分钟' if QUICK_TEST_MODE else '1-3小时'}")
print("-" * 60)

training_results = trainer.train(train_loader, val_loader)

print("\n" + "=" * 60)
print(f"✅ {'快速测试' if QUICK_TEST_MODE else '完整'}训练完成！")
print(f"🏆 最佳模型: {training_results['best_model_path']}")
print(f"📈 最佳验证损失: {training_results['best_val_loss']:.6f}")
print(f"⏱️  训练轮次: {len(training_results['train_losses'])}")

if QUICK_TEST_MODE:
    print("\n💡 快速测试模式结果说明:")
    print("   - 此结果仅用于验证代码逻辑")
    print("   - 模型性能可能不是最优的")
    print("   - 建议设置 QUICK_TEST_MODE = False 进行完整训练")

## 7. 模型评估

In [ ]:
# 加载最佳模型进行评估
print("📂 加载最佳训练模型...")

# 优先从训练器获取最佳模型路径，其次从训练结果获取
best_model_path = getattr(trainer, 'best_model_path', None) or training_results.get('best_model_path')
if best_model_path and os.path.exists(best_model_path):
    print(f"🎯 最佳模型路径: {best_model_path}")
    
    # 加载最佳模型权重
    try:
        # 加载完整的检查点
        checkpoint = torch.load(best_model_path, map_location=device)
        
        # 检查是否包含model_state_dict键
        if 'model_state_dict' in checkpoint:
            portfolio_model.load_state_dict(checkpoint['model_state_dict'])
            print("✅ 最佳模型加载成功!")
            print(f"  训练轮次: {checkpoint.get('epoch', 'Unknown')}")
            print(f"  验证损失: {checkpoint.get('val_loss', 'Unknown'):.6f}")
        else:
            # 直接加载state_dict（兼容旧格式）
            portfolio_model.load_state_dict(checkpoint)
            print("✅ 最佳模型加载成功!")
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        print("🔄 使用当前模型状态进行评估...")
else:
    print("⚠️  未找到最佳模型路径，使用当前模型状态进行评估")

print("\n🔍 开始在测试集上评估模型...")

# 在测试集上评估模型
portfolio_model.eval()
test_predictions = []
test_losses = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        weights = portfolio_model(batch_X)
        loss = loss_fn(weights, batch_y)
        test_predictions.append(weights.cpu())
        test_losses.append(loss.item())

# 合并预测结果
test_predictions = torch.cat(test_predictions, dim=0).numpy()
test_returns = data_dict['y_test'].cpu().numpy()

# 计算投资组合收益
portfolio_returns = np.sum(test_predictions * test_returns, axis=1)

print(f"\n📊 最佳模型测试集评估结果:")
print(f"  测试集损失: {np.mean(test_losses):.6f}")
print(f"  投资组合收益统计:")
print(f"    平均收益: {np.mean(portfolio_returns):.6f}")
print(f"    收益波动: {np.std(portfolio_returns):.6f}")
print(f"    夏普比率: {np.mean(portfolio_returns) / np.std(portfolio_returns):.6f}")

# 计算更多投资组合指标
cumulative_return = np.sum(portfolio_returns)
max_return = np.max(portfolio_returns)
min_return = np.min(portfolio_returns)
positive_returns = np.sum(portfolio_returns > 0)
total_returns = len(portfolio_returns)
win_rate = positive_returns / total_returns

print(f"\n💰 详细投资组合分析:")
print(f"    累积总收益: {cumulative_return:.6f}")
print(f"    最大单期收益: {max_return:.6f}")
print(f"    最小单期收益: {min_return:.6f}")
print(f"    胜率: {win_rate:.2%} ({positive_returns}/{total_returns})")

# 显示权重分布统计
print(f"\n⚖️  权重分配分析:")
mean_weights = np.mean(test_predictions, axis=0)
for i, (ticker, weight) in enumerate(zip(tickers, mean_weights)):
    print(f"    {ticker}: {weight:.4f} ({weight*100:.2f}%)")

# 验证权重约束
weights_sum_check = np.abs(np.sum(test_predictions, axis=1) - 1.0)
max_weight_deviation = np.max(weights_sum_check)
print(f"\n✅ 权重约束验证:")
print(f"    权重和最大偏差: {max_weight_deviation:.8f}")
print(f"    权重和约束满足: {'✅ 是' if max_weight_deviation < 1e-4 else '❌ 否'}")

In [ ]:
# 🏋️ 高级模型训练过程
def train_advanced_attention_model(model, train_loader, val_loader, config):
    """训练高级多层注意力投资组合模型"""
    
    if model is None:
        print("❌ 无法训练：模型为空。请先成功创建模型。")
        return None, None, None
        
    print("? 开始训练高级多层注意力投资组合模型...")
    
    # 创建训练器
    try:
        from xiaojiucai.utils.training_utils import ModelTrainer, TrainingConfig
        from xiaojiucai.losses.portfolio_loss import PortfolioLoss
        
        # 创建训练配置
        training_config = TrainingConfig()
        training_config.portfolio_lr = 1e-4  # 较小的学习率用于稳定训练
        training_config.timesfm_lr = 1e-5   # TimesFM使用更小的学习率
        training_config.patience = 15
        training_config.num_epochs = 50
        training_config.finetune_timesfm = False  # 开始时冻结
        
        # 创建损失函数
        portfolio_loss = PortfolioLoss(
            risk_aversion=training_config.risk_aversion,
            sharpe_weight=training_config.sharpe_weight
        )
        
        # 创建训练器
        advanced_trainer = ModelTrainer(
            model=model,
            config=training_config,
            loss_fn=portfolio_loss,
            device=device,
            model_save_dir='models'
        )
        
    except ImportError as e:
        print(f"❌ 无法导入训练工具: {e}")
        print("💡 请确保xiaojiucai包已正确安装")
        return None, None, None
    
    # 训练配置
    train_config = {
        'epochs': 100,
        'log_interval': 5,
        'save_checkpoint': True,
        'checkpoint_path': 'models/advanced_attention_portfolio_model.pth',
        'enable_attention_visualization': True
    }
    
    # 检查是否在快速测试模式
    if 'QUICK_TEST_MODE' in globals() and QUICK_TEST_MODE:
        # 快速测试模式：减少训练轮数
        train_config['epochs'] = 5
        print("⚡ 快速测试模式：减少训练轮数至5轮")
    
    # 第一阶段：冻结TimesFM训练
    print("🔒 第一阶段：冻结TimesFM，训练注意力头...")
    
    # 确保TimesFM参数冻结
    for param in model.timesfm_model.parameters():
        param.requires_grad = False
    
    # 训练
    try:
        # 更新训练配置中的轮数
        training_config.num_epochs = min(50, train_config['epochs'])  # 第一阶段最多50轮
        
        result = advanced_trainer.train(train_loader, val_loader)
        
        train_losses = result['train_losses'] if 'train_losses' in result else []
        val_losses = result['val_losses'] if 'val_losses' in result else []
        
        print("✅ 第一阶段训练完成！")
    except Exception as e:
        print(f"❌ 第一阶段训练失败: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None
    
    # 第二阶段：解冻TimesFM进行端到端微调（可选）
    fine_tune_timesfm = False  # 默认不进行微调，避免复杂性
    
    if 'QUICK_TEST_MODE' not in globals() or not QUICK_TEST_MODE:
        # 只在完整训练模式下询问
        try:
            response = input("🤔 是否进行第二阶段端到端微调？(y/n): ").lower()
            fine_tune_timesfm = response == 'y'
        except:
            # 如果无法获取用户输入（例如在自动化环境中），默认不微调
            fine_tune_timesfm = False
            print("⚡ 自动跳过第二阶段微调")
    else:
        print("⚡ 快速测试模式：跳过第二阶段微调")
    
    if fine_tune_timesfm:
        print("🔓 第二阶段：解冻TimesFM，端到端微调...")
        
        # 解冻TimesFM参数
        for param in model.timesfm_model.parameters():
            param.requires_grad = True
            
        model.finetune_timesfm = True
        
        # 继续训练
        try:
            # 更新配置以支持TimesFM微调
            training_config.finetune_timesfm = True
            training_config.num_epochs = min(30, train_config['epochs']//2)  # 第二阶段训练轮数
            training_config.timesfm_lr = 1e-6  # 更小的学习率用于微调
            training_config.portfolio_lr = 1e-5  # 稍微降低portfolio层学习率
            
            # 重新创建训练器以使用新的优化器设置
            advanced_trainer_stage2 = ModelTrainer(
                model=model,
                config=training_config,
                loss_fn=portfolio_loss,
                device=device,
                model_save_dir='models'
            )
            
            result_2 = advanced_trainer_stage2.train(train_loader, val_loader)
            
            train_losses_2 = result_2['train_losses'] if 'train_losses' in result_2 else []
            val_losses_2 = result_2['val_losses'] if 'val_losses' in result_2 else []
            
            # 合并损失历史
            train_losses.extend(train_losses_2)
            val_losses.extend(val_losses_2)
            
            print("✅ 第二阶段端到端微调完成！")
        except Exception as e:
            print(f"❌ 第二阶段训练失败: {e}")
            import traceback
            traceback.print_exc()
    
    # 保存最终模型
    if train_config['save_checkpoint']:
        try:
            # 确保模型目录存在
            os.makedirs('models', exist_ok=True)
            
            # 获取资产名称
            if 'tickers' in globals():
                asset_names = tickers
            else:
                asset_names = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
            
            torch.save({
                'model_state_dict': model.state_dict(),
                'config': config,
                'train_losses': train_losses,
                'val_losses': val_losses,
                'asset_names': asset_names
            }, train_config['checkpoint_path'])
            print(f"💾 模型已保存至: {train_config['checkpoint_path']}")
        except Exception as e:
            print(f"⚠️  模型保存失败: {e}")
    
    return train_losses, val_losses, advanced_trainer

# 检查训练的前置条件
print("🔍 检查训练的前置条件...")
required_vars = ['train_loader', 'val_loader', 'device']
missing_vars = []

for var in required_vars:
    if var not in globals():
        missing_vars.append(var)

if missing_vars:
    print(f"⚠️  缺少训练所需变量: {missing_vars}")
    print("📝 需要先运行数据加载单元格来创建 train_loader 和 val_loader")
    print()
    print("💡 训练函数已定义，但需要在所有前置条件满足后才能运行")
else:
    print("✅ 训练前置条件检查完成")
    
    # 只有在模型和数据都准备好时才尝试训练
    if 'advanced_model' in globals() and advanced_model is not None:
        print("🏃‍♂️ 准备开始高级注意力模型训练...")
        print(f"📊 训练集大小: {len(train_loader.dataset)}")
        print(f"📊 验证集大小: {len(val_loader.dataset)}")
        print(f"🎯 目标: 学习复杂的跨资产关系和时间依赖模式")
        
        # 询问是否立即开始训练
        try:
            start_training = input("🤔 是否立即开始训练？(y/n): ").lower() == 'y'
        except:
            start_training = False
            print("⚡ 自动跳过训练，可以稍后手动调用训练函数")
        
        if start_training:
            # 执行训练
            train_losses_advanced, val_losses_advanced, advanced_trainer = train_advanced_attention_model(
                advanced_model, train_loader, val_loader, model_config
            )
            
            if train_losses_advanced is not None:
                print("🎉 高级注意力模型训练完成！")
            else:
                print("❌ 训练失败")
        else:
            print("📝 训练函数已定义，可以稍后使用以下代码手动启动:")
            print("train_losses_advanced, val_losses_advanced, advanced_trainer = train_advanced_attention_model(advanced_model, train_loader, val_loader, model_config)")
    else:
        print("⚠️  模型尚未创建，无法开始训练")
        print("📝 训练函数已定义，等待模型创建完成后使用")

## 8. 可视化分析

In [ ]:


# 绘制训练曲线
plot_training_curves(training_results['train_losses'], training_results['val_losses'])

# 创建综合分析报告
portfolio_metrics, individual_metrics = create_comprehensive_report(
    test_predictions, portfolio_returns, test_returns, tickers
)

## 🔧 快速测试模式说明

本notebook支持两种运行模式：

### ⚡ 快速测试模式 (`QUICK_TEST_MODE = True`)
- **数据量**: 500个时间点（**最新数据！**）
- **数据范围**: 2023-2025年（当前市场环境）
- **时间窗口**: 100个时间步
- **批量大小**: 32
- **训练轮次**: 3轮
- **预计时间**: 5-10分钟
- **用途**: 快速验证代码逻辑，调试流程

### ? 完整训练模式 (`QUICK_TEST_MODE = False`)
- **数据量**: 完整数据集（2005-2025年）
- **时间窗口**: 200个时间步
- **批量大小**: 128
- **训练轮次**: 20轮
- **预计时间**: 1-3小时
- **用途**: 完整模型训练，获得最佳性能

### ? 重要改进：使用最新数据

快速测试模式现在使用数据集的**后500行**（最新数据），而不是前500行：

- **旧方式**: 2005-2007年数据，AAPL平均~$2
- **新方式**: 2023-2025年数据，AAPL平均~$207  
- **优势**: 更相关的测试结果，更好的调试体验

💡 **建议**: 首次运行时使用快速测试模式验证流程，确认无误后再切换到完整模式进行正式训练。

## 🛠️ TypeError 修复总结

### 问题描述
遇到了 `TypeError: ModelTrainer.__init__() got an unexpected keyword argument 'learning_rate'` 错误。

### 🔍 问题分析
检查 `xiaojiucai.utils.training_utils.ModelTrainer` 类发现：
- 该类需要 `TrainingConfig` 对象，而不是直接的学习率参数
- 需要显式传递 `loss_fn` (损失函数)
- `train` 方法返回字典而不是元组

### ✅ 修复措施

1. **正确的 ModelTrainer 初始化**：
   ```python
   # ❌ 错误的方式
   ModelTrainer(model=model, device=device, learning_rate=1e-4)
   
   # ✅ 正确的方式
   training_config = TrainingConfig()
   portfolio_loss = PortfolioLoss()
   ModelTrainer(model=model, config=training_config, loss_fn=portfolio_loss, device=device)
   ```

2. **正确的训练调用**：
   ```python
   # ❌ 错误的方式
   train_losses, val_losses = trainer.train(train_loader, val_loader, epochs=50)
   
   # ✅ 正确的方式
   result = trainer.train(train_loader, val_loader)
   train_losses = result['train_losses']
   val_losses = result['val_losses']
   ```

3. **两阶段训练优化**：
   - 第一阶段：冻结TimesFM，只训练注意力头
   - 第二阶段：解冻TimesFM，端到端微调（使用更小学习率）

### 🎯 修复结果
- ✅ ModelTrainer 参数错误已修复
- ✅ 训练配置正确设置
- ✅ 损失函数正确传递
- ✅ 两阶段训练逻辑完善
- ✅ 错误处理和异常捕获增强

现在可以正常运行训练过程了！